<table align="left">
  <td>
    <a href="https://colab.research.google.com/drive/1lm1hHlrVNkEz5QgpaiuNBPR93Ja1Y7HR" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
  </td>
</table>

---

# **Licencia**

**Autor**: Juan Francisco Puentes Calvo

**Licencia**: GPL v3 (https://www.gnu.org/licenses/gpl-3.0.html)


# **Reconocimientos**

* Niklas Fiekas: https://github.com/niklasf/python-chess
* Inspirado en: Anthony Sanchez: https://github.com/AnthonyASanchez/python-chess-ai

---

# **¿Jugamos al ajedrez?**



In [1]:
!pip install -q jupyter_ui_poll
!pip install -q chess


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 5.1 MB/s eta 0:00:00


El **algoritmo Minimax** es una técnica de decisión utilizada en la teoría de juegos, la inteligencia artificial para encontrar el movimiento óptimo para un jugador, asumiendo que el oponente también está jugando de manera óptima, en un contexto de **información perfecta**. Se aplica en juegos de dos jugadores con **suma cero**, lo que significa que la ganancia de un jugador es exactamente la pérdida del otro, como en el caso del ajedrez, damas, tic-tac-toe, entre otros.

El funcionamiento básico del algoritmo Minimax se puede describir en los siguientes pasos:

1. **Generar el árbol de juego**: El algoritmo construye un árbol de juego que representa todas las posibles posiciones o estados del juego hasta una cierta profundidad, partiendo del estado actual. Cada nodo del árbol representa un estado del juego, y las ramas representan los movimientos posibles desde ese estado.

2. **Evaluar los nodos hoja**: Para los nodos en el nivel más profundo del árbol (nodos hoja), se asigna un valor evaluativo que indica qué tan favorable es ese estado para el jugador. Este valor puede venir de una *función de evaluación* específica del juego.

3. **Aplicar la recursión Minimax**: El algoritmo utiliza una función recursiva para navegar a través del árbol desde los nodos hoja hacia el nodo raíz. En cada paso, el algoritmo selecciona el movimiento óptimo utilizando el siguiente criterio:

 * Si el nodo es un turno del jugador maximizador (el que intenta maximizar el resultado), el algoritmo escoge el valor máximo de entre los valores de los nodos hijos.
 * Si el nodo es un turno del jugador minimizador (el que intenta minimizar el resultado), se escoge el valor mínimo de entre los valores de los nodos hijos.
 * Seleccionar el mejor movimiento: Una vez que el algoritmo ha evaluado el nodo raíz, elige el movimiento (o los movimientos) que conducen al mejor valor evaluativo posible, asumiendo una estrategia óptima por parte del oponente.


In [2]:
#@title [**Algoritmo MINIMAX**](https://es.wikipedia.org/wiki/Minimax)

import math;
import random;
import sys;

#-----------------------------------------------------------------------
def minimax(depth, board, α=-2500, β=+2500):
    rootTurn=board.turn;
    bestMoveScore = float("-inf");
    bestMoveFinal = None;
    for x in board.legal_moves:
        move = chess.Move.from_uci(str(x));
        board.push(move);
        try:
          value = max(bestMoveScore, _minimax(depth-1, board, α, β, rootTurn));
          if value > bestMoveScore:
             bestMoveScore = value;
             bestMoveFinal = move;
        finally:
          board.pop();
    return bestMoveFinal;

#-----------------------------------------------------------------------
def _minimax(depth, board, α, β, rootTurn):

    if depth == 0:

       v=evaluateBoard(board);
       if board.turn == rootTurn:
          return v[int(board.turn)]-v[int(not board.turn)];
       else:
          return v[int(not board.turn)]-v[int(board.turn)];

    if board.turn == rootTurn:

       bestMove = float("-inf");
       for x in board.legal_moves:
           move = chess.Move.from_uci(str(x));
           board.push(move);
           try:
             bestMove = max(bestMove, _minimax(depth - 1, board, α, β, rootTurn));
             α = max(α, bestMove);
             if β <= α: return bestMove;
           finally:
             board.pop();

       return bestMove;

    else:

        bestMove = float("+inf");
        for x in board.legal_moves:
            move = chess.Move.from_uci(str(x));
            board.push(move);
            try:
              bestMove = min(bestMove, _minimax(depth - 1, board, α, β, rootTurn));
              β = min(β,bestMove);
              if β <= α: return bestMove;
            finally:
              board.pop();

        return bestMove

#-----------------------------------------------------------------------
def evaluateBoard(board):
    """
    Recorre el tablero, calculando la evaluación del mismo.
    Devuelve una lista con la suma de los valores de las piezas negras (0) y las blancas (1).
    """
    evaluation = [0,0];
    for i in range(64):
        piece = board.piece_at(i);
        if piece is not None and piece.color==chess.BLACK: evaluation[0] += getPieceValue(str(piece));
        if piece is not None and piece.color==chess.WHITE: evaluation[1] += getPieceValue(str(piece));
    return evaluation;

#-----------------------------------------------------------------------
def getPieceValue(piece):
    """
    Establece el valor de cada figura del ajedrez.
    Modificando estos valores, modificamos el comportamiento en el juego.
    """
    if piece == "P" or piece == "p": return 10;   # peón
    if piece == "N" or piece == "n": return 450;  # caballo
    if piece == "B" or piece == "b": return 250;  # alfil
    if piece == "R" or piece == "r": return 200;  # torre
    if piece == "Q" or piece == "q": return 800;  # reina
    if piece == 'K' or piece == 'k': return 900;  # rey
    return 0;


In [3]:
#@title **Clase CHESS**

import chess;

import random;
import time;

from jupyter_ui_poll import ui_events;
from IPython.display import display, update_display, HTML;
from ipywidgets      import widgets;

class Chess:

      def __init__(self, pause=0, depth=3, width=630):
          self._board=chess.Board();
          self._pause=pause;
          self._depth=depth;

      def who(self, player):
          return "Blancas" if player == chess.WHITE else "Negras";

      def show(self, uci=None):
          #clear_output(wait=True);
          name = self.who(self._board.turn);
          board_stop = self._board._repr_svg_();
          if uci: html = "<b>Movimiento %s %s, Jugada '%s':</b><br/>%s" % (len(self._board.move_stack), name, uci, board_stop);
          else:   html = "<b>Movimiento %s %s:</b><br/>%s"              % (len(self._board.move_stack), name,      board_stop);
          update_display(HTML(html), display_id="BOARD");

      def random_player(self):
          move = random.choice(list(self._board.legal_moves));
          return move.uci();

      def human_player(self):
          move=None;

          def on_move_change(change):
              nonlocal move;
              move=change["new"].uci();

          def on_random_move(w):
              nonlocal move;
              move=self.random_player();

          def on_minimax_move(w):
              nonlocal move;
              move=self.minimax_player(self._depth);

          lm=sorted(self._board.legal_moves, key=lambda x: x.uci());
          move_widget=widgets.Dropdown(options=lm, description='Jugadas legales posibles:', disabled=False);
          move_widget.observe(on_move_change,"value");
          random_widget=widgets.Button(description="Aleatorio", style={'button_color': '#FFD700'});
          random_widget.on_click(on_random_move);
          minimax_widget=widgets.Button(description="Minimax", style={'button_color': '#FFD700'});
          minimax_widget.on_click(on_random_move);
          hbox=widgets.HBox([move_widget, random_widget, minimax_widget]);
          display(hbox, display_id="MOVE");
          with ui_events() as poll:
              while move is None:
                    poll(10);
                    time.sleep(0.1);
          move_widget.close();
          del move_widget;
          random_widget.close();
          del random_widget;
          minimax_widget.close();
          del minimax_widget;
          return move;

      def minimax_player(self):
          move = minimax(self._depth,self._board);
          return move.uci();

      def play(self, player1, player2):
          try:
              self.show();
              while not self._board.is_game_over(claim_draw=False):
                    if self._board.turn == chess.WHITE: uci = player1();
                    else:                               uci = player2();
                    self._board.push_uci(uci);
                    self.show(uci);
                    time.sleep(PAUSE);

          except KeyboardInterrupt:
              msg = "Juego interruptido!";
              return (None, msg, self._board);

          if   self._board.is_checkmate():
               msg = "Jaque mate: " + self.who(not self._board.turn) + " ganan!";
          elif self._board.is_stalemate():
               msg = "Empate: tablas por imposibilidad de hacer un movimiento legal.";
          elif self._board.is_fivefold_repetition():
               msg = "Empate: por repetición (5) de posición.";
          elif self._board.is_insufficient_material():
               msg = "Empate: por carecer de potencial para ganar.";
          elif self._board.can_claim_draw():
               name = self.who(self._board.turn);
               msg = f"'{name}' piden tablas.";

          return (msg, self._board);


In [ ]:
#@title **Jugadores aleatorios de ajedrez**

#@markdown Segundos transcurridos entre jugadas

PAUSE = 1 #@param {"type":"slider", min:0.1, max:2, step:0.1}

game = Chess(pause=PAUSE);
game.play(game.random_player, game.random_player)

In [ ]:
#@title **Jugador aleatorio de ajedrez contra un humano**

#@markdown El jugador humano empieza con blancas.

game = Chess();
game.play(game.human_player, game.random_player);

In [ ]:
#@title **Jugador inteligente de ajedrez contra un humano**

#@markdown El agente inteligente usa el [algoritmo minimax](https://es.wikipedia.org/wiki/Minimax).

#@markdown El jugador humano empieza con blancas..

PAUSE = 1 #@param {"type":"slider", min:0.1, max:2, step:0.1}

DEPTH = 3 #@param {"type":"slider", min:1, max:5, step:1}

game = Chess(depth=DEPTH, pause=PAUSE);
game.play(game.human_player, game.minimax_player)